In [1]:
!pip install mplfinance

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install settrade-v2

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install psycopg2


Defaulting to user installation because normal site-packages is not writeable


In [4]:
from settrade_v2 import Investor
from pprint import pprint
import psycopg2
import pandas as pd
import time
from psycopg2 import sql
from datetime import datetime
from pprint import pprint
import mplfinance as mpf
import matplotlib.pyplot as plt
import numpy as np

In [5]:
investor = Investor(
                    app_id="K2IdT8f5M1JOjszq",
                    app_secret="AJ5gSbLc2HOLhYQxQgXNft4iv1488hp3BgNUvFo/ceDo",
                    broker_id="SANDBOX",
                    app_code="SANDBOX",
                    is_auto_queue = False)

In [6]:
# กรณี Investor
market = investor.MarketData()

In [7]:
mkt_data = investor.MarketData()
res = mkt_data.get_quote_symbol("CPALL")
pprint(res)

{'aumSize': None,
 'average': 45.92437,
 'change': -1.5,
 'eps': 1.57,
 'exchange': None,
 'exercisePrice': None,
 'exerciseRatio': None,
 'high': 46.0,
 'impliedVolatility': None,
 'inav': None,
 'instrumentType': 'STOCK',
 'intrinsicValue': None,
 'last': 44.5,
 'lastTradingDate': None,
 'low': 44.5,
 'marketStatus': 'Open1',
 'maturityDate': None,
 'moneyness': None,
 'pbv': 3.36,
 'pe': 15.71,
 'percentChange': -3.26,
 'percentYield': 2.84,
 'securityType': 'CS',
 'status': '',
 'symbol': 'CPALL',
 'theoretical': None,
 'toLastTrade': None,
 'totalBuyVolume': 300,
 'totalNoSideVolume': 11000,
 'totalSellVolume': 600,
 'totalVolume': 11900,
 'underlying': None,
 'underlyingPrice': None}


In [9]:
from settrade_v2 import Investor
from settrade_v2.logger import SettradeError

try:
    mkt_data = investor.MarketData()
    res = mkt_data.get_quote_symbol("CPALL")
    print(res)
except SettradeError as e:
    print("SettradeError code:", getattr(e, "code", None))
    print("SettradeError message:", getattr(e, "message", str(e)))
except Exception as e:
    print("Other error:", e)


{'instrumentType': 'STOCK', 'symbol': 'CPALL', 'high': 46.0, 'low': 44.5, 'last': 44.5, 'average': 45.92437, 'change': -1.5, 'percentChange': -3.26, 'totalVolume': 11900, 'totalBuyVolume': 300, 'totalSellVolume': 600, 'totalNoSideVolume': 11000, 'status': '', 'marketStatus': 'Open1', 'securityType': 'CS', 'eps': 1.57, 'pe': 15.71, 'pbv': 3.36, 'percentYield': 2.84, 'maturityDate': None, 'exercisePrice': None, 'underlying': None, 'underlyingPrice': None, 'intrinsicValue': None, 'theoretical': None, 'moneyness': None, 'lastTradingDate': None, 'toLastTrade': None, 'exerciseRatio': None, 'impliedVolatility': None, 'exchange': None, 'aumSize': None, 'inav': None}


In [10]:
db_config = {
    'dbname': 'settrade_api',
    'user': 'postgres',
    'password': '111004',
    'host': 'localhost',
    'port': "5432"
}

In [ ]:
# ✅ ฟังก์ชันสร้างตาราง
def create_table():
    create_table_query = """
    CREATE TABLE IF NOT EXISTS stock_financials (
        symbol TEXT PRIMARY KEY,
        instrument_type TEXT DEFAULT 'STOCK',
        security_type TEXT DEFAULT 'CS',
        market_status TEXT DEFAULT 'Unknown',
        total_volume INT DEFAULT 0,
        aum_size NUMERIC DEFAULT 0,
        average NUMERIC DEFAULT 0,
        change NUMERIC DEFAULT 0,
        eps NUMERIC DEFAULT 0,
        exchange TEXT DEFAULT '',
        exercise_price NUMERIC DEFAULT 0,
        exercise_ratio NUMERIC DEFAULT 0,
        high NUMERIC DEFAULT 0,
        implied_volatility NUMERIC DEFAULT 0,
        inav NUMERIC DEFAULT 0,
        intrinsic_value NUMERIC DEFAULT 0,
        last NUMERIC DEFAULT 0,
        last_trading_date DATE,
        low NUMERIC DEFAULT 0,
        maturity_date DATE,
        moneyness NUMERIC DEFAULT 0,
        pbv NUMERIC DEFAULT 0,
        pe NUMERIC DEFAULT 0,
        percent_change NUMERIC DEFAULT 0,
        percent_yield NUMERIC DEFAULT 0,
        theoretical NUMERIC DEFAULT 0,
        to_last_trade NUMERIC DEFAULT 0,
        underlying TEXT DEFAULT '',
        underlying_price NUMERIC DEFAULT 0,
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """
    try:
        # 🔗 เชื่อมต่อฐานข้อมูล
        with psycopg2.connect(**db_config) as conn:
            with conn.cursor() as cursor:
                cursor.execute(create_table_query)
                conn.commit()
        print("✅ ตาราง `stock_financials` ถูกสร้างหรือมีอยู่แล้ว!")
    except Exception as e:
        print("❌ Error creating table:", e)

# ✅ รันฟังก์ชันสร้างตาราง
create_table()


In [11]:
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import os
import time
import psycopg2
import pandas as pd
from settrade_v2 import Investor
from dotenv import load_dotenv

# ✅ โหลดตัวแปรจาก .env
load_dotenv()


# ✅ สร้าง Object สำหรับ Market Data
market = investor.MarketData()

# ✅ ฟังก์ชันดึงข้อมูล Financial Data และบันทึกลง PostgreSQL
def insert_financial_data(symbol):
    try:
        print(f"🔍 กำลังดึงข้อมูล Financial Data สำหรับ {symbol}...")
        
        # ✅ ดึงข้อมูลจาก Settrade API
        res = market.get_quote_symbol(symbol)  # 🔥 เปลี่ยนจาก get_quote() เป็น get_quote_symbol()

        # 🚨 ถ้าไม่มีข้อมูล ให้ข้าม
        if not isinstance(res, dict) or "symbol" not in res:
            print(f"⚠️ ไม่มีข้อมูลสำหรับ {symbol}.")
            return

        # 🔧 กำหนดค่าเริ่มต้นหากไม่มีข้อมูล
        data = {
            'symbol': res.get('symbol', symbol),
            'instrument_type': res.get('instrumentType', 'STOCK'),
            'security_type': res.get('securityType', 'CS'),
            'market_status': res.get('marketStatus', 'Unknown'),
            'total_volume': res.get('totalVolume', 0),
            'aum_size': res.get('aumSize', 0),
            'average': res.get('average', 0),
            'change': res.get('change', 0),
            'eps': res.get('eps', 0),
            'exchange': res.get('exchange', ''),
            'exercise_price': res.get('exercisePrice', 0),
            'exercise_ratio': res.get('exerciseRatio', 0),
            'high': res.get('high', 0),
            'implied_volatility': res.get('impliedVolatility', 0),
            'inav': res.get('inav', 0),
            'intrinsic_value': res.get('intrinsicValue', 0),
            'last': res.get('last', 0),
            'last_trading_date': res.get('lastTradingDate', None),
            'low': res.get('low', 0),
            'maturity_date': res.get('maturityDate', None),
            'moneyness': res.get('moneyness', 0),
            'pbv': res.get('pbv', 0),
            'pe': res.get('pe', 0),
            'percent_change': res.get('percentChange', 0),
            'percent_yield': res.get('percentYield', 0),
            'theoretical': res.get('theoretical', 0),
            'to_last_trade': res.get('toLastTrade', 0),
            'underlying': res.get('underlying', ''),
            'underlying_price': res.get('underlyingPrice', 0)
        }

        # ✅ เชื่อมต่อฐานข้อมูลและบันทึกข้อมูล
        with psycopg2.connect(**db_config) as conn:
            with conn.cursor() as cursor:
                insert_query = """
                    INSERT INTO stock_financials (
                        symbol, instrument_type, security_type, market_status, total_volume, 
                        aum_size, average, change, eps, exchange, exercise_price, 
                        exercise_ratio, high, implied_volatility, inav, intrinsic_value, 
                        last, last_trading_date, low, maturity_date, moneyness, pbv, pe, 
                        percent_change, percent_yield, theoretical, to_last_trade, underlying, 
                        underlying_price, updated_at
                    ) VALUES (
                        %(symbol)s, %(instrument_type)s, %(security_type)s, %(market_status)s, %(total_volume)s,
                        %(aum_size)s, %(average)s, %(change)s, %(eps)s, %(exchange)s, %(exercise_price)s,
                        %(exercise_ratio)s, %(high)s, %(implied_volatility)s, %(inav)s, %(intrinsic_value)s,
                        %(last)s, %(last_trading_date)s, %(low)s, %(maturity_date)s, %(moneyness)s, %(pbv)s, %(pe)s,
                        %(percent_change)s, %(percent_yield)s, %(theoretical)s, %(to_last_trade)s, %(underlying)s,
                        %(underlying_price)s, NOW()
                    )
                    ON CONFLICT (symbol) DO UPDATE SET
                        last = EXCLUDED.last,
                        change = EXCLUDED.change,
                        percent_change = EXCLUDED.percent_change,
                        high = EXCLUDED.high,
                        low = EXCLUDED.low,
                        total_volume = EXCLUDED.total_volume,
                        updated_at = NOW();
                """
                cursor.execute(insert_query, data)
                conn.commit()
                print(f"✅ ข้อมูล Financial Data ของ {symbol} ถูกบันทึกเรียบร้อยแล้ว!")

    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาดกับ {symbol}: {e}")

# ✅ โหลดรายชื่อหุ้นทั้งหมดจากไฟล์ CSV
try:
    symbols_df = pd.read_csv(r"D:\Document\DSA\DSA 3\DSA3_1-2568\Data Mining\\thai_stocks.csv")  # ไฟล์ต้องมีคอลัมน์ 'symbol'
    symbols = symbols_df['symbol'].dropna().tolist()
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ 'thai_stocks.csv' ใช้รายชื่อหุ้นที่กำหนดแทน")
    symbols = ["PTT", "AOT", "SCB", "CPALL", "ADVANC"]

# ✅ ดึงข้อมูล Financial Data สำหรับหุ้นทุกตัว
for symbol in symbols:
    insert_financial_data(symbol)
    time.sleep(2)  # ⏳ ควบคุม API Rate Limit

In [12]:
import pandas as pd

# ✅ อ่านแค่ header ของไฟล์
file_path = r"D:\Document\DSA\DSA 3\DSA3_1-2568\Data Mining\thai_stocks.csv"
symbols_df = pd.read_csv(file_path, nrows=0)  # อ่านแค่ header
print("Columns ในไฟล์ CSV:")
print(symbols_df.columns.tolist())


Columns ในไฟล์ CSV:
['symbol']


In [13]:
print(symbols_df.columns.tolist())

['symbol']
